# Agentic Workflows

## Grading Lab: Using Validation in Multi-Agent Pipelines

In this mini-exam, you will implement a small multi-agent workflow that produces a short research report.  
Your **Planner Agent** must return a compact plan with a maximum of 4 steps, and one of those steps must explicitly begin with `[VALIDATE]`.  

The **validation step** checks whether a sufficient number of URLs come from **trusted domains**.  

You are free to adjust the topic, the minimum validation threshold, and the trusted domain list.

In [7]:
import aisuite
import ast
import urllib.parse
from dotenv import load_dotenv

_ = load_dotenv()  # Load environment variables from a .env file if present
client = aisuite.Client()


MIN_VALIDATION_RATIO = 0.4  # Minimum ratio of trusted sources to total results

TOP_DOMAINS = {}

TOPIC="The life cycle of a butterfly"

In [8]:
def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Planner Agent: calls an LLM to return a compact list of steps.
    Must include a literal step that starts with [VALIDATE].
    Max 4 steps total.
    """
    user_prompt = f"""
    Return ONLY a valid Python list of strings (list[str]).
    Maximum 4 steps.
    The steps MUST include:
      1) A Tavily web search for info on '{topic}' (return JSON with title,url,source)
      2) A validation step that STARTS WITH "[VALIDATE]"
      3) Draft a short Markdown report
      4) Final report

    Topic: "{topic}"
    """.strip()

    ### START CODE HERE ###
    resp = client.chat.completions.create( # @KEEP client.chat.completions.create(
        model=model,  # @REPLACE model=None,
        messages=[{"role": "user", "content": user_prompt}], # @REPLACE messages=None,
        temperature=1, # @REPLACE temperature=None,
    )
    ### END CODE HERE ###

    steps_str = resp.choices[0].message.content.strip()
    return ast.literal_eval(steps_str)  # students will debug if parsing fails


In [9]:
### START CODE HERE ###

def validate_tavily_results(results: list[str], min_ratio: float = 0.4) -> str:
    """Check if enough results are from trusted domains."""
    def hostname(url: str) -> str:
        host = urllib.parse.urlparse(url).hostname or ""
        return host[4:] if host.startswith("www.") else host

    total = len(results)
    approved = sum(1 for r in results if any(dom in hostname(r) for dom in TOP_DOMAINS))  # @REPLACE approved = None
    ratio = approved / max(total, 1) # @REPLACE ratio = None
    status = "PASS ✅" if ratio >= min_ratio else "FAIL ⚠️" # @REPLACE status = None
    return f"Validation: {status} ({approved}/{total} = {ratio:.0%})"

### END CODE HERE ###


In [10]:
def research_agent(task: str) -> str:
    return f"[Research completed for: {task}]"

def writer_agent(task: str) -> str:
    return f"[Draft created for: {task}]"

def editor_agent(task: str) -> str:
    return f"[Edited version of: {task}]"


In [11]:
def executor_agent(plan: list[str]) -> list[tuple[str, str]]:
    history = []
    for step in plan:
        if step.startswith("[VALIDATE]"):
            mock_results = [
                "https://en.wikipedia.org/wiki/Nova",
                "https://www.fashionnova.com/"
            ]
            out = validate_tavily_results(mock_results, min_ratio=MIN_VALIDATION_RATIO) # @REPLACE validate_tavily_results(None, None)
        elif "Search" in step:
            out = research_agent(step) 
        elif "Draft" in step:
            out = writer_agent(step) # @REPLACE out = None
        elif "Final" in step:
            out = editor_agent(step) # @REPLACE out = None
        else:
            out = "Unknown step"
        history.append((step, out))
    return history


In [12]:
plan = planner_agent(TOPIC)
print("Plan:", plan)

history = executor_agent(plan)
print("\nExecution:")
for step, out in history:
    print(f"- {step}: {out}")


Plan: ["A Tavily web search for info on 'The life cycle of a butterfly' (return JSON with title, url, source)", '[VALIDATE] Verify the retrieved data for accuracy, completeness, and reliability against authoritative scientific sources', 'Draft a short Markdown report summarizing the four stages of a butterfly’s life cycle with headings and concise descriptions', "Final report: present the completed Markdown document on 'The life cycle of a butterfly'"]

Execution:
- A Tavily web search for info on 'The life cycle of a butterfly' (return JSON with title, url, source): Unknown step
- [VALIDATE] Verify the retrieved data for accuracy, completeness, and reliability against authoritative scientific sources: Validation: FAIL ⚠️ (0/2 = 0%)
- Draft a short Markdown report summarizing the four stages of a butterfly’s life cycle with headings and concise descriptions: [Draft created for: Draft a short Markdown report summarizing the four stages of a butterfly’s life cycle with headings and conci